 # Домашнее задание 2. Нейронные сети

### Овсепян Ара ИУ8-83

В зависимости от варианта, для решения задачи используйте следующие наборы данных:

<table>
    <thead>
         <tr>
            <th colspan=1>Задача классификации</th>
            <th colspan=5>Задача регрессии</th>
        </tr>
        <tr>
            <th> </th>
            <th>A</th>
            <th>B</th>
            <th>C</th>
            <th>D</th>
            <th>E</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>K</td>
            <td>1</td>
            <td>2</td>
            <td>3</td>
            <td>4</td>
            <td>5</td>
        </tr>
        <tr>
            <td>L</td>
            <td>6</td>
            <td>7</td>
            <td>8</td>
            <td style="background-color: green">9</td>
            <td>10</td>
        </tr>
        <tr>
            <td>M</td>
            <td>11</td>
            <td>12</td>
            <td>13</td>
            <td>14</td>
            <td>15</td>
        </tr>
        <tr>
            <td>N</td>
            <td>16</td>
            <td>17</td>
            <td>18</td>
            <td>19</td>
            <td>20</td>
        </tr>
        <tr>
            <td>O</td>
            <td>21</td>
            <td>22</td>
            <td>23</td>
            <td>24</td>
            <td>25</td>
        </tr>
    </tbody>
</table>

## 0. Инициализация ноутбука


Загрузка необходимых библиотек

In [ ]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

from torch import FloatTensor
from torch.nn import Module

from torch import allclose
from torch import from_numpy

from math import sqrt                       
from torch.nn.functional import mse_loss 

from torch.nn import Sequential
from torch.nn import Linear

from torch.nn import Sigmoid
from torch.nn import Tanh
from torch.nn import ReLU
from torch.nn import ELU

from torch.optim import SGD
from torch.optim import RMSprop
from torch.optim import Adam

from torch.nn import BatchNorm1d
from torch.nn import Dropout

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

## 1. Искусственная неронная сеть для решения задачи регрессии (9 баллов)

In [4]:
BASELINE = 5800
emoloyeeAttrition = pd.read_csv('../data/Employee-Attrition.csv')

### 1.1 Загрузка данных


Загрузите данные в датафрейм при помощи функции `read_csv` блиблиотеки pandas

In [5]:
data = pd.read_csv('./data/insurance.csv')

### 1.2 График совместного распределения признаков


Постройте график совместного распределения количественных признаков при помощи функции pairplot библиотеки seaborn

In [ ]:
sns.pairplot(data)

### 1.3 Разделение набора данных на тренировочный и тестовый

Разделите набор данных на тренировочный и тестовый в соотношении 80%/20% при помощи функции `train_test_split` библиотеки `sklearn.model_selection`. При необходимости категориальные признаки в числовые (например, при помощи функции `get_dummies` библиотеки pandas)

In [7]:
Train, Test = train_test_split(pd.get_dummies(data), train_size=0.8,)

## 1.1 Загрузите данные, поделить на train и test

In [8]:
insurances = pd.read_csv('./data/insurance.csv', sep=',')
insurances = pd.get_dummies(insurances)

In [ ]:
Y = insurances.charges
X = insurances.drop(columns='charges')

X_scaled = StandardScaler(copy=True, with_mean=True, with_std=True).fit_transform(X)
X = pd.DataFrame(X_scaled, index=X.index, columns=X.columns)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=69)

print(X_train.head())
print(Y_train.head())

### 1.2 По необходимости проведите преодбработку/нормализацию данных

In [ ]:
X_t_train = FloatTensor(X_train.values)
X_t_test  = FloatTensor(X_test.values )
Y_t_train = FloatTensor(Y_train.values)
Y_t_test  = FloatTensor(Y_test.values )

### 1.3 Попробуйте различное количество слоёв
Обучите 4 модели, используя различное количество линейных слоёв: 1 (=линейная регрессия), 2, 3, 4. Между двумя линейными слоями должна быть фунция активации, например, сигмоида. На последнем слое функцию активации можно не использовать.

Постройте: 1) график loss для train и test в зависимости от используемого количетсва слоёв; 2) график accuracy для train и test в зависимости от используемого количества слоёв.

In [ ]:
_e = 0.0001 

In [ ]:
class Model(Module):
    def __init__(self, X, Y, a, l, f=ReLU, opt=Adam, lr=0.3, momentum=None):
        super(Model, self).__init__()
        self.X = X
        self.y = Y
        self.attrs = a
        self.amount = l
        self.activ = f

        self.batch = None
        self.dropout = None

        self.gen_layers()
        self.opt = opt(self.conv.parameters(), lr=lr)
        if momentum:
            self.opt = opt(self.conv.parameters(), lr=lr, momentum=momentum)

    def gen_layers(self):
        self.conv = Sequential()

        for i in range(self.amount):
            if i == self.amount - 1:
                self.conv.add_module(f'lin_{i+1}', Linear(self.attrs, 1))
                continue
            
            self.conv.add_module(f'lin_{i+1}', Linear(self.attrs, self.attrs))
            self.normalized(i)
            self.conv.add_module(f'act_{i+1}', self.activ())

    def normalized(self, i):
        if self.dropout is not None:
            self.conv.add_module(f'Drp_{i+1}', Dropout(p=self.dropout))

        if self.batch is not None:
            self.conv.add_module(f'Bth_{i+1}', BatchNorm1d(self.attrs))
            
    def batch_norm(self):
        self.batch = True

    def dropout(self, a):
        self.dropout = a

    def net(self,  X=None):
        if X is None:
            X = self.X
            
        return self.conv.forward(X)[:,0]

    def loss(self, pred, Y=None):
        if Y is None:
            Y = self.y
        return mse_loss(pred, Y)

    def train(self, epochs=1000):
        loss = []
        accur = []
        for _ in range(epochs):
            self.opt.zero_grad()
            self.conv.train()

            pred = self.net()

            loss = self.loss(pred)
            loss.backward()

            self.opt.step()
            
            loss.append(sqrt(loss.item()))

            accuracy = (pred == self.y).float().sum()
            accur.append(accuracy * 100 / pred.shape[0])

            plt.yscale('log')

        plt.plot(loss)
        plt.show()
        plt.plot(accur)
        plt.show()

        return accur
    
    def test(self, X, Y):
        pred = self.net(X)
        loss = self.loss(pred, Y)
        ls = sqrt(loss.item())

        print(f'Loss: {ls}')
        
        return ls

In [11]:
for i in range(4):
    model = Model(X_t_train, Y_t_train, len(X_train.columns), i+1, f=ReLU)
    print(model)
    model.train()
    model.test(X_t_test, Y_t_test)
    print()

### 1.4 Попробуйте различные функции активации
Обучите 4 модели, используя 4 различных функции активации: sigmoid, tanh, ReLU, ELU.

Постройте: 1) график loss для train и test в зависимости от используемой функции активации; 2) график accuracy для train и test в зависимости от используемой функции активации;

In [ ]:
model = Model(X_train, Y_train, len(X_train.columns), 4, Sigmoid)
print(model)
model.train()
model.test(X_t_test, Y_t_test)

In [ ]:
model = Model(X_t_train, Y_t_train, len(X_train.columns), 4, Tanh)
print(model)
model.train()
model.test(X_t_test, Y_t_test)

In [ ]:
model = Model(X_t_train, Y_t_train, len(X_train.columns), 4, ReLU)
print(model)
model.train()
model.test(X_t_test, Y_t_test)

In [ ]:
model = Model(X_t_train, Y_t_train, len(X_train.columns), 4, ELU)
print(model)
model.train()
model.test(X_t_test, Y_t_test)

## 1.5 Попробуйте различные алгоритмы оптимизации

Обучите 4 модели, используя 4 различных алгоритма оптимизации: SGD, SGD with momentum, RMSprop, Adam

Постройте: 1) график loss для train и test в зависимости от используемого алгоритма оптимизации; 2) график accuracy для train и test в зависимости от используемого алгоритма оптимизации.

In [ ]:
model = Model(X_t_train, Y_t_train, len(X_train.columns), 4, ReLU, SGD, lr=0.000001)
print(model)
model.train()
model.test(X_t_test, Y_t_test)

In [ ]:
model = Model(X_t_train, Y_t_train, len(X_train.columns), 4, ReLU, SGD, lr=0.0001, momentum=0.04)
print(model)
model.train()
model.test(X_t_test, Y_t_test)

In [ ]:
model = Model(X_t_train, Y_t_train, len(X_train.columns), 4, ReLU, RMSprop)
print(model)
model.train()
model.test(X_t_test, Y_t_test)

In [ ]:
model = Model(X_t_train, Y_t_train, len(X_train.columns), 4, ReLU, Adam)
print(model)
model.train()
model.test(X_t_test, Y_t_test)

## 1.6 Попробуйте добавить dropout и batch norm

Обучите 5 моделей, используя 5 различных конфигураций: только BatchNorm, только dropout 0.2, только dropout 0.5, BatchNorm + dropout 0.2, BatchNorm + dropout 0.5.

Постройте: 1) график loss для train и test в зависимости от используемой конфигурации; 2) график accuracy для train и test в зависимости от используемой конфигурации.

In [ ]:
model = Model(X_t_train, Y_t_train, len(X_train.columns), 4)
model.batch_norm()
model.gen_layers()
print(model)
model.train()
model.test(X_t_test, Y_t_test)

In [ ]:
model = Model(X_t_train, Y_t_train, len(X_train.columns), 4)
model.dropout(0.2)
model.gen_layers()
print(model)
model.train()
model.test(X_t_test, Y_t_test)

In [ ]:
model = Model(X_t_train, Y_t_train, len(X_train.columns), 4)
model.dropout(0.5)
model.gen_layers()
print(model)
model.train()
model.test(X_t_test, Y_t_test)

In [ ]:
model = Model(X_t_train, Y_t_train, len(X_train.columns), 4)
model.batch_norm()
model.dropout(0.2)
model.gen_layers()
print(model)
model.train()
model.test(X_t_test, Y_t_test)

In [ ]:
model = Model(X_t_train, Y_t_train, len(X_train.columns), 4)
model.batch_norm()
model.dropout(0.5)
model.gen_layers()
print(model)
model.train()
model.test(X_t_test, Y_t_test)

## 1.7 Постройте финальную модель
Используя информацию, полученную на предыдущих этапах, предложите оптимальную конфигурацию нейронной сети для вашей задачи. Обучите модель.

In [25]:
model = Model(X_t_train, Y_t_train, len(X_train.columns), 4, ReLU)
print(model)
model.train()
model.test(X_t_test, Y_t_test)

In [26]:
res = fun(train, Train.charges, test, Test.charges)
res_p = fun(train_proc, Train_proc.charges, test_proc, Test_proc.charges)

In [ ]:
plt.grid()

plt.xlabel('Alpha')
plt.ylabel('RMSE error')

plt.plot(res[0], res[1], marker='o')
plt.plot(res_p[0], res_p[1], marker='o')

plt.show()

## 1.8 Вывод

Сравните качество (величину ошибки) для моделей из пп. 1.4-1.7. Какая модель показала наилучший результат?

Для всех 3 типов моделей RMSE ошибка ниже порога Baseline только при исключении выбросов (> 77 процентиля) из набора данных.

Сама же ошибка во всех трех случаях практически идентична (см. ниже).

Из за возможности настройки параметров `alpha` и `solver` в `Ridge` и `Lasso`, следует предположить, что при других наборах параметров, эти методы дадут лучший резульат по сравнению с `LinearRegression`, из за отсутствия настройки таких доп. параметров.

In [ ]:
print('Model\t\tRegular\t\tWith excluding\n')
for m, e in RES.items():
    print(f'{m}\t\t{round(e[0], 2)}\t\t{round(e[1], 2)}')

# 2. Искусственная неронная сеть для решения задачи классификации (9 баллов)

## 2.1 Загрузите данные, поделить на train и test

In [ ]:
BASELINE = 0.70
employee_Attrition = pd.read_csv('../data/Employee-Attrition.csv')

## 2.2 По необходимости проведите преодбработку/нормализацию данных

In [ ]:
dummy_col = ['BusinessTravel','Department','EducationField','Gender','JobRole','MaritalStatus','Over18','OverTime']
dummy_attrition = pd.get_dummies(employee_Attrition, columns=dummy_col)

dummy_attrition['Attrition'] = LabelEncoder().fit_transform(dummy_attrition['Attrition'])

normal_attrition = pd.DataFrame(
    StandardScaler().fit(dummy_attrition).transform(dummy_attrition),
     columns=dummy_attrition.columns)

train, test, train_attrition, test_attrition = train_test_split(
    normal_attrition,
    dummy_attrition.Attrition,
    train_size=0.8,
    random_state=1
)

train_d = train.drop(columns='Attrition')
test_d = test.drop(columns='Attrition')

## 2.3 Попробуйте различное количество слоёв
Обучите 4 модели, используя различное количество линейных слоёв: 1 (=логистическая регрессия), 2, 3, 4. Между двумя линейными слоями должна быть фунция активации, например, сигмоида. На последнем слое в качестве функции активации используйте сигмоиду.

Постройте: 1) график loss для train и test в зависимости от используемого количетсва слоёв; 2) график accuracy для train и test в зависимости от используемого количества слоёв.

In [ ]:
def gen_layers(num, activationFunc= None):
    layers = []

    for layer in range(num):
        if layer == num - 1:
            layers.append(nn.Linear(55, 2))
            layers.append(nn.LogSoftmax(dim=1))
        else:
            layers.append(nn.Linear(55, 55))
            layers.append(activationFunc)
    
    return layers

In [38]:
def get_loss(droppedValues, values, model):
    predict = model.forward(torch.FloatTensor(droppedValues))
    loss = functional.nll_loss(predict, torch.LongTensor(values))

    return predict, loss

In [39]:
def score(data, predict):
    return accuracy_score(data,np.argmax(predict.detach().numpy(), axis=1))

92

In [ ]:
def train(model, optimizer, e):
    loss_val = []
    loss_val_test = []

    acc_val = []
    acc_val_test = []

    for i in range(e):
        optimizer.zero_grad()
        model.train() 

        predict, loss = get_loss(test_d.values, train_attrition.values, model)
        acc_val.append(score(train_attrition, predict))

        predict_test, loss_test = get_loss(test_d.values, test_attrition.values, model)
        loss_val_test.append(loss_test.item())
        acc_val_test.append(score(test_attrition, predict_test))

        loss.backward()
        optimizer.step()

        loss_val.append(loss.item())

    plt.plot(loss_val)
    plt.plot(loss_val_test)
    plt.yscale('log')
    plt.show()

    plt.plot(acc_val_test)
    plt.yscale('log')
    plt.show()

In [ ]:
def train_test_model(model, optimizer, epochs=1000):
    train(model, optimizer, epochs)
    test(model)

In [ ]:
def test(model):
    predict, loss = get_loss(test_d.values, test_attrition.values, model)

    result = np.argmax(predict.detach().numpy(), axis=1)
    f1 = f1_score(test_attrition, result, average='macro')
    
    print(classification_report(test_attrition, result, zero_division=0))
    print(f'f1_score: {f1}')
    print(f'loss: {loss.item()}')

### 2.3.1 Слои

#### Слой 1

In [40]:
layers = gen_layers(1)
model = nn.Sequential(*layers)

train_test_model(model, torch.optim.Adam(model.parameters(), lr=0.001))

### Слой 2

In [41]:
layers = gen_layers(2, nn.ReLU())
model = nn.Sequential(*layers)

train_test_model(model, torch.optim.Adam(model.parameters(), lr=0.001))

### Слой 3

In [ ]:
layers = gen_layers(3, nn.ReLU())
model = nn.Sequential(*layers)

train_test_model(model, torch.optim.Adam(model.parameters(), lr=0.001))

### Слой 4

In [ ]:
layers = gen_layers(4, nn.ReLU())
model = nn.Sequential(*layers)

train_test_model(model, torch.optim.Adam(model.parameters(), lr=0.001))

## 2.4 Попробуйте различные функции активации
Обучите 4 модели, используя 4 различных функции активации на промежуточных слоях: sigmoid, tanh, ReLU, ELU. На выходном слое в качестве функции активации используйте сигмоиду.

Постройте: 1) график loss для train и test в зависимости от используемой функции активации; 2) график accuracy для train и test в зависимости от используемой функции активации;

### ReLU

In [62]:
layers = gen_layers(4, nn.ReLU())
model = nn.Sequential(*layers)

train_test_model(model, torch.optim.Adam(model.parameters(), lr=0.001))

### ELU

In [ ]:
layers = gen_layers(4, nn.ELU())
model = nn.Sequential(*layers)

train_test_model(model, torch.optim.Adam(model.parameters(), lr=0.001))

### Sigmoid

In [ ]:
ayers = gen_layers(4, nn.Sigmoid())
model = nn.Sequential(*layers)

train_test_model(model, torch.optim.Adam(model.parameters(), lr=0.001))

### Tanh

In [ ]:
layers = gen_layers(4, nn.Tanh())
model = nn.Sequential(*layers)

train_test_model(model, torch.optim.Adam(model.parameters(), lr=0.001))

## 2.5 Попробуйте различные алгоритмы оптимизации
Обучите 4 модели, используя 4 различных алгоритма оптимизации: SGD, SGD with momentum, RMSprop, Adam

Постройте: 1) график loss для train и test в зависимости от используемого алгоритма оптимизации; 2) график accuracy для train и test в зависимости от используемого алгоритма оптимизации.

### SGD

In [ ]:
layers = gen_layers(4, nn.Sigmoid())
model = nn.Sequential(*layers)

train_test_model(model, torch.optim.SGD(model.parameters(), lr=0.001))

### SGD with momentum

In [ ]:
layers = gen_layers(4, nn.Sigmoid())
model = nn.Sequential(*layers)

train_test_model(model, torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.5))

### RMSprop

In [ ]:
layers = gen_layers(4, nn.Sigmoid())
model = nn.Sequential(*layers)

train_test_model(model, torch.optim.RMSprop(model.parameters(), lr=0.001))

### Adam

In [ ]:
layers = gen_layers(4, nn.Sigmoid())
model = nn.Sequential(*layers)

train_test_model(model, torch.optim.Adam(model.parameters(), lr=0.001))

## 2.6 Попробуйте добавить dropout и batch norm

Обучите 5 моделей, используя 5 различных конфигураций: только BatchNorm, только dropout 0.2, только dropout 0.5, BatchNorm + dropout 0.2, BatchNorm + dropout 0.5.

Постройте: 1) график loss для train и test в зависимости от используемой конфигурации; 2) график accuracy для train и test в зависимости от используемой конфигурации.

In [ ]:
def dbn(num, func = None, batch=55, dropout=0):
    layers = []
    for layer in range(num):
        if layer == num - 1:
            layers.append(nn.Linear(55, 2))
            layers.append(nn.LogSoftmax(dim=1))
        else:
            layers.append(nn.Linear(55, 55))
            if batch:
                layers.append(nn.BatchNorm1d(batch))
            if dropout:
                layers.append(nn.Dropout(p=dropout))
            layers.append(func)
    
    return layers

### BatchNorm

In [ ]:
layers = dbn(4, nn.Sigmoid())
model = nn.Sequential(*layers)

train_test_model(model, torch.optim.Adam(model.parameters(), lr=0.001))

### Dropout 

In [ ]:
layers = dbn(4, nn.Sigmoid(), 0, 0.2)
model = nn.Sequential(*layers)

train_test_model(model, torch.optim.Adam(model.parameters(), lr=0.001))

### Dropout 

In [ ]:
layers = dbn(4, nn.Sigmoid(), 0, 0.5)
model = nn.Sequential(*layers)

train_test_model(model, torch.optim.Adam(model.parameters(), lr=0.001))

### BatchNorm + Dropout

In [ ]:
layers = dbn(4, nn.Sigmoid(), dropout=0.2)
model = nn.Sequential(*layers)

train_test_model(model, torch.optim.Adam(model.parameters(), lr=0.001))

In [ ]:
layers = dbn(4, nn.Sigmoid(), dropout=0.5)
model = nn.Sequential(*layers)

train_test_model(model, torch.optim.Adam(model.parameters(), lr=0.001))

## 2.7 Постройте финальную модель
Используя информацию, полученную на предыдущих этапах, предложите оптимальную конфигурацию нейронной сети для вашей задачи. Обучите модель.

In [ ]:
layers = gen_layers(4, nn.Sigmoid())
model = nn.Sequential(*layers)

train_test_model(model, torch.optim.Adam(model.parameters(), lr=0.001))

## 2.8 Выводы
4-ех слойная модель с алгоритмом оптимизации Adam и функцией активацией Sigmoid показала самую высокую точность